---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [2]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [118]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    
#     from io import StringIO
#     uni_towns = pd.read_csv(StringIO(data))
#     uni_towns = pd.read_csv(
#         'university_towns.txt',
#         header='1'
#         header=[0,10]
#         delim_whitespace=True,
#         colspecs=[(0,5), (5, 2000)]
#         delimiter=' '
#     )
#     uni_towns = pd.read_csv('university_towns.txt', delimiter= '\s+', index_col=False)
#     uni_towns = pd.read_fwf(
#         'university_towns.txt',
# #         header='1'
# #         header=[0,10]
# #         delim_whitespace=True,
# #         colspecs=[(0,5), (5, 2000)]
# #         delimiter=' '
#     )

#     uni_towns = pd.read_csv(
#         'university_towns.txt',
#         delimiter = "\t",
#         chunksize=1,
#     )
    
    df = pd.read_table('university_towns.txt',header = None)
#     result = []
    d = {'State': [], 'RegionName': []}
    res = pd.DataFrame(data=d)
    
    states_set = set(states.values())
#     print(states_set)
#     state = str(df.iloc[0].values)[2:-8]
    state = ''
    for i in range(0,len(df)):
        try:
            val = df.iloc[i]
            val = val.str.replace('\[(.*?)\]', '')
            val = val.str.replace('\(.*', ' ')
            val = val.str.replace('\s+', '')[0]

            if val in states_set:
                state = val
            else:
#                 print(res.DataFrame([state, val], columns=['State', 'RegionName']))
                res = res.append({'State': state, 'RegionName': val}, ignore_index=True)
#                 res.concat(
#                     [res.DataFrame([state, val], columns=['State', 'RegionName'])],
#                       ignore_index=True
#                 )
#                 res = res.append([state, val], ignore_index=True)
#             print(val.values)
#             if val.str.contains('\('):
#                 # not a state
#                 print('jere', val.str)
# #             else:
#                 #state
#                 state = val.str
#                 print('state:{}'.format(state))
#             print(val.str.contains('\('))
# #             if val.str.contains('('):
# #                 print(val)
        except Exception:
            pass
#             print(val.str.contains('[').bool())
#         if val.contains('[edit]'):
#             print(val)
#         val = df.iloc[i].values[0]
#         if val.
#         startswith
#         print(df.iloc[i].values[0])

#     uni_towns.columns = ["a", "b", "c", "etc."]
#     print(uni_towns.iloc[6])
#     for p in uni_towns:
#         print(p.iloc[0])

#     res = res.set_index(['State','RegionName'])
    return res

get_list_of_university_towns()

,State,RegionName
0,Alabama,Auburn
1,Alabama,Florence
2,Alabama,Jacksonville
3,Alabama,Livingston
4,Alabama,Montevallo
...,...,...
519,Wisconsin,RiverFalls
520,Wisconsin,StevensPoint
521,Wisconsin,Waukesha
522,Wisconsin,Whitewater


In [4]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    
    GDP = pd.read_excel(
        'gdplev.xls',
        usecols=[4,5],
        header=[5],
        skiprows=2,
        index_col=0
    )

    GDP = GDP.loc['2000q1':,:]
    GDP.rename(columns={'Unnamed: 5': 'q_gdp'}, inplace=True)
    GDP['tmp'] = (GDP['q_gdp'] < GDP['q_gdp'].shift(-1))
    GDP['tmp2'] = (GDP['tmp'] == False) & (GDP['tmp'].shift(-1) == False)
    
    return GDP.iloc[np.where(GDP['tmp2'] == True)].iloc[0].name
#     GDP.reset_index()

#     print(GDP.head(40))
#     GDP['tmp'] = np.where(
#         [GDP['q_gdp'] < GDP['q_gdp'].shift(-1)]
#     )
#     print(GDP)
#     print(GDP)
#     GDP.rename(columns={'GDP in billions of current dollars.1': 'gdp'}, inplace=True)
#     GDP = GDP.set_index('year_quarter')
#     GDP['growth_direction'] = True if (GDP.gdp.eq(GDP.gdp.shift()) >= 0) else False


#     GDP['Position1'] = np.where(
#         (
#             GDP['gdp'] < GDP['gdp'].shift(-1)
#         )
#         ,
#         True,
#         False
#     )


#     GDP['Position2'] = np.where(
#         (
#             (GDP['gdp'].shift(-1) < GDP['gdp'].shift(-2))
#         )
#         ,
#         True,
#         False
#     )

#     x = (GDP['gdp'] < GDP['gdp'].shift(-1))
#     x = x['2006q1':]
#     mask = (GDP['gdp'].shift(-1) >= GDP['gdp'].shift(-2))['2006q1':]
#     mask = mask.where(mask == False, inplace=True)
    
#     x.rename(columns={'year_quarter', 'asdas'}, inplace=True)

#     filtered = np.where(
#         [GDP['Position1'] == False & GDP['Position2'] == False]
#     )
    
#     return GDP.iloc[filtered[1][0]].name
#     print(GDP[GDP['Position1'] == False & GDP['Position2'] == False])

#     print(GDP)
#     ngative_growth_quarters = np.where(GDP['Position'] == True)
#     return GDP.iloc[ngative_growth_quarters[0][0]].name

get_recession_start()

'2008q3'

In [105]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    
    recession_start = get_recession_start()

    GDP = pd.read_excel(
        'gdplev.xls',
        usecols=[4,5],
        header=[5],
        skiprows=2,
        index_col=0
    )

    GDP = GDP.loc[recession_start:,:]
    GDP.rename(columns={'Unnamed: 5': 'q_gdp'}, inplace=True)
#     print(GDP)
    GDP['tmp'] = (GDP['q_gdp'] > GDP['q_gdp'].shift(-1))
#     GDP['tmp2'] = (GDP['tmp'] == False) & (GDP['tmp'].shift(-1) == False)
    return GDP.iloc[np.where(GDP['tmp'] == False)].iloc[2:,:].iloc[0].name

get_recession_end()

'2009q4'

In [153]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3
    A recession bottom is the quarter within a recession which had the lowest GDP.'''
    
    recession_start = get_recession_start()
    recession_end = get_recession_end()

    GDP = pd.read_excel(
        'gdplev.xls',
        usecols=[4,5],
        header=[5],
        skiprows=2,
        index_col=0
    )

    GDP = GDP.loc[recession_start:recession_end,:]
    GDP.rename(columns={'Unnamed: 5': 'q_gdp'}, inplace=True)
    return GDP.idxmin()['q_gdp']

get_recession_bottom()

'2009q2'

In [157]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    homes = pd.read_csv(
        'homes.csv',
    )
#     homes = homes.drop_duplicates(subset=['RegionName'])
#     homes = homes.drop_duplicates(subset=['State'])

    homes = homes[['RegionName','State'] + list(homes.columns[56:257])]
    homes.drop_duplicates(['RegionName','State'], inplace=True)
    homes.replace({"State": states}, inplace=True)
#     homes = homes['State'].map(states)
    print(type(homes))
    homes.set_index(['RegionName','State'], inplace=True)
    homes = homes.dropna()
    
#     print(cols)
#     cols = cols[:-1]
#     pd.rename?
#     homes.rename(columns=cols, inplace=True)
#     print(homes.head())
    
#     homes = homes.index.drop_duplicates()
#     print(homes)
#     a = list(range(3,51))
#     homes.drop(homes.columns[a],axis=1,inplace=True)

#     homes.set_index(['State', 'RegionName'], inplace=True)
#     homes = homes.iloc[:,:6]
#     homes = homes.iloc[:,:6], homes.iloc[:,7:9]]

#     print(homes.iloc[:2,1:6])
#      homes = homes['State'+'RegionName' + list(homes.columns[56:257])]
#     homes = homes[list(homes.columns[:6]) + list(homes.columns[56:257])]
#     print(homes.size)
#     homes.drop_duplicates(inplace=True)
#     print(homes.size)
#     homes_1 = homes.iloc[:,2:]
#     print(homes_1.head(2))
    years = 17
    q_per_y = 4
    m_per_q = 3

    total_group_count = years * q_per_y * m_per_q - m_per_q
    quarter_mask = np.arange(total_group_count)//3
#     print(quarter_mask.size, homes_1.size)
#     print(homes_1.shape)
#     print(homes_1.head())
#     print(quarter_mask)
    grouped = homes.groupby(quarter_mask, axis=1).mean()
    grouped.sort_index(inplace=True)
    
    grouped.columns = [
        str(2000+i)+'q'+str(j)
        for i in range(17)
        for j in range(1,5)
    ][:-1]
    
#     grouped.iloc[:,:3].to_csv('tmp.csv')

#     for i in range(17):
#         for j in range(1,5):
#             print(str(2000+i)+'q'+str(j))
#     print(grouped.shape)
#     for x in range(grouped.size):
#         print(grouped.iloc[x])
#         if x in xx:
#             print('dup')
#         else:
#             xx[x] = y
#     print(grouped.head())
#     grouped.replace({'State':states}, inplace = True)
#     print(grouped.iloc[13350:])
#     print(grouped.index[13300:])
#     transformed = grouped.transform(lambda x: x.fillna(x.mean()))
#     print(grouped.iloc[20:30])
#     print()
#     print(homes['RegionName'].iloc[:20])
#     x = homes[homes['State','RegionName']] + grouped
#     x = get_list_of_university_towns()
#     print(x)
#     print(transformed.head(8))
#     homes = homes[list(homes.columns[2:2]) + list(homes.columns[6:12])]
#     homes = homes[:2, 3:6]
#     print(x)

    return grouped


convert_housing_data_to_quarters()

<class 'pandas.core.frame.DataFrame'>


2000q1         2000q2         2000q3  \
RegionName    State                                                         
Abbeville     Alabama          46354.666667   47571.666667   48219.000000   
              Georgia          37754.333333   37126.000000   36747.666667   
              South Carolina   74296.666667   74184.000000   74601.000000   
Abbottstown   Pennsylvania    110443.666667  110746.666667  112320.666667   
Abell         Maryland        192120.000000  194202.666667  197805.333333   
...                                     ...            ...            ...   
Zolfo Springs Florida          67797.000000   67545.000000   68782.666667   
Zumbro Falls  Minnesota       176232.666667  181902.000000  179549.333333   
Zumbrota      Minnesota       114058.666667  117036.333333  121242.333333   
Zuni          Virginia        107732.333333  109493.000000  112867.666667   
Zwolle        Louisiana        95852.000000   97086.333333   96089.000000   

                                     2000q4         2001q1         2001q2  \
RegionName    State                                                         
Abbeville     Alabama          47987.000000   48830.000000   49521.000000   
              Georgia          37091.333333   37895.666667   38427.333333   
              South Carolina   75399.000000   76889.666667   77711.666667   
Abbottstown   Pennsylvania    114589.666667  115723.000000  116560.000000   
Abell         Maryland        198691.000000  204009.000000  204972.333333   
...                                     ...            ...            ...   
Zolfo Springs Florida          71349.000000   73258.666667   75546.666667   
Zumbro Falls  Minnesota       176755.333333  180431.333333  184326.000000   
Zumbrota      Minnesota       124992.666667  127374.333333  130058.666667   
Zuni          Virginia        115758.333333  119013.000000  121508.666667   
Zwolle        Louisiana        97451.000000   99169.666667  101514.000000   

                                     2001q3         2001q4         2002q1  \
RegionName    State                                                         
Abbeville     Alabama          49568.666667   50117.000000   50744.666667   
              Georgia          39466.333333   40151.000000   41106.666667   
              South Carolina   78747.333333   79980.000000   81109.333333   
Abbottstown   Pennsylvania    117499.666667  118326.333333  119869.000000   
Abell         Maryland        207082.000000  216746.000000  223107.666667   
...                                     ...            ...            ...   
Zolfo Springs Florida          77379.666667   78499.000000   79996.000000   
Zumbro Falls  Minnesota       188423.666667  188542.666667  194938.333333   
Zumbrota      Minnesota       133438.333333  136748.000000  140048.000000   
Zuni          Virginia        123586.000000  125431.000000  128118.666667   
Zwolle        Louisiana       108335.333333  111537.666667  112109.666667   

                                     2002q2  ...         2014q2  \
RegionName    State                          ...                  
Abbeville     Alabama          51956.333333  ...   74878.333333   
              Georgia          42879.333333  ...   49788.333333   
              South Carolina   82264.333333  ...  101832.333333   
Abbottstown   Pennsylvania    121434.333333  ...  156153.666667   
Abell         Maryland        233317.666667  ...  289837.666667   
...                                     ...  ...            ...   
Zolfo Springs Florida          80942.666667  ...   97485.666667   
Zumbro Falls  Minnesota       195351.666667  ...  226557.666667   
Zumbrota      Minnesota       143269.666667  ...  160432.666667   
Zuni          Virginia        132395.333333  ...  194812.666667   
Zwolle        Louisiana       116390.333333  ...   92276.333333   

                                     2014q3         2014q4         2015q1  \
RegionName    State                                                         
Abbev

In [165]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    housing_data_quarters = convert_housing_data_to_quarters()
    
#     new_data = housing_data_quarters.loc[:, [before_recession, recession_bottom]].reset_index()
#     new_data = housing_data_quarters.loc[:,recession_start:recession_bottom]
#     new_data['Change'] = new_data['2009q2'] / new_data['2008q3']
#     print(new_data)
    
    # Create university towns dummy
    u_towns = get_list_of_university_towns()
    u_towns = u_towns[['State', 'RegionName']]
    u_towns['U_Town'] = 1
    u_towns = u_towns.set_index(['State', 'RegionName'])
    
    # Merge university towns with housing prices by state and region name
    x = convert_housing_data_to_quarters()
    
    x = x.merge(u_towns, how = 'left', left_index = True, right_index = True)
#     print(np.where(x['U_Town'] == 1))
    x.U_Town[x.U_Town.isnull()] = 0
#     print(x[x['U_Town'] == 1].sum()
#     print(x)
    
    start = get_recession_start()
    bottom = get_recession_bottom()
    y = x[[start, bottom, 'U_Town']]
    y['Change'] = y['2009q2'] / y['2008q3']

    ut = y[y['U_Town'] == 1]['Change'].dropna()
    not_ut = y[y['U_Town'] == 0]['Change'].dropna()
    t = ttest_ind(ut, not_ut)
    p = t[1]
    
    different = False
    if p < 0.01:
        different = True

    better = 'university town'
    if ut.mean() < not_ut.mean():
        better = 'non-university town'

    return (different, p, better)

run_ttest()

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>


/Users/mani/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


(False, 0.9051572433997666, 'non-university town')